In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
!pip install langchain==0.0.224

In [ ]:
!pip install boto3 --upgrade

In [ ]:
!pip install ~/SageMaker/botocore-1.29.162-py3-none-any.whl
!pip install ~/SageMaker/boto3-1.26.162-py3-none-any.whl

### Pre-requisites

Ensure you have access to Bedrock and the Claude models

In [ ]:
import boto3
import langchain
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain import SagemakerEndpoint, LLMChain
from langchain import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.prompts import StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException

import re

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


### Create the tools

We create the "tools" which will be executed by agent. In this example, we create the a tool to generate the fictious TPS reports made famous by the move Office Space. The LLM would parse the input from the user and determine that the tool has to been invoked with the parameter provided by the user. The generateTpsReport tool, returns the query from the user along with asking the LLM to generate a paragraph about the topic the user specified. This is a simplistic example, however this method could easily be modified to lookup a database or invoke and external service.

In [ ]:
import requests
from langchain.tools import BaseTool
from bs4 import BeautifulSoup
import random

def generateTpsReport(topic):
    print("\nthe tps report tool executed\n")
    return "<function_result>here is a report about " + topic + ". Write 5 sentences in a paragraph about " + topic + " as a official status report </function_result>"


tps_report_tool = Tool(
    name="generateTpsReport",
    func = generateTpsReport,
    description="a tool to generate tps reports"
)

tools = [tps_report_tool]

### The ReAct prompt

A prompt template is created to feed the LLM and it includes an example for the LLM to understand how process the inputs. 


In [ ]:
claude_template = """Human: You are a research assistant AI that has been equipped with the following function(s) to help you answer a <question>. Your goal is to answer the user's question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. The result of a function call will be added to the conversation history as an observation.

"Here are the only function(s) I have provided you with:

{tools}

Human: "To call a function, output <function_call>insert specific function</function_call>. You will receive a <function_result> in response to your call that contains information that you can use to better answer the question. You should not geneate a function_result, it should be provided to you

Here is an example of how you would correctly answer a question using a <function_call> and the corresponding <function_result>. Notice that you are free to think before deciding to make a <function_call> in the <scratchpad>:

<example>
<functions>
<function>
<function_name>generateTpsReport</function_name>
<function_description>Generates a TPS report</function_description>
<required_argument>report_subject(string): the topic the report needs to be generated about </required_argument>
<returns>string: A document containting the report.</returns>
<raises>ValueError: if a valid topic is not provided </raises>
<example_call>generate_tps_reports(topic=""red_stapler"")</example_call>
</function>
</functions>

<question>Generate a TPS report about staplers?</question>

<scratchpad>I do not have access to generate random numbers so I should use a function to gather more information to answer this question. I have been equipped with the function get_random_number that gets a random number  so I should use that to gather more information.

I have double checked and made sure that I have been provided the generate_tps_reports function.
</scratchpad>

<function_call>generateTpsReport(topic=""stapler"")</function_call>

<function_result>Milton has a red stapler</function_result>

<answer>Milton has a red stapler</answer>
</example>"

<question>{input}</question>

{agent_scratchpad}

Assistant:

"""

### The Prompt Template

A custom prompt template is defined which will insert the actual data in the location of the macros in the prompt ```{input}``` and ```{tool}```

In [ ]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\n\n{observation}"
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        #tools = self.tools_getter(kwargs["input"])
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a tools variable from the list of tools provided
        #kwargs["tools"] = "\n".join(
        #    [f"{tool.name}: {tool.description}" for tool in tools]
        #)
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        #print("the kwargs in the format function before returning is")
        #print(kwargs)
        return self.template.format(**kwargs)

claude_prompt = CustomPromptTemplate(
    template=claude_template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

### Output Parser

The agent invokes the Output parser once it receives a response from the LLM. The Output parser is responsible for examining the output from the LLM and determining the tool to use and the parameters to send to the tool. This class determines the next action an agent should take, which could either be AgentFinish or AgentAction. 

In the prompt, we have instructed the LLM to answer with the ```<answer>``` XML tag once it has all the information for the answer, hence we look for the ```<answer>``` tag in the LLM output and if present, we extract the answer, and wrap it in an AgentFinish object and return it. When the agent receives the AgentFinish object it stops the execution of the agent and provides the answer to the end-user

If the prompt has ```<function_call>``` we return an object of AgentAction that contains the name of the function to invoke and the parameters to pass to the function. The agent will use the information from this object to invoke the tool and pass the response back to the LLM

In [ ]:
import xml.etree.ElementTree as ET

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:

        # Check if agent should finish
        if "<answer>" in llm_output:
            #print("Agent is finished")
            root = ET.fromstring(llm_output)
            output = root.text

            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
               return_values={"output": output},
                log=llm_output,
            )
        
        
        linesInOutput = llm_output.splitlines()
        #print(len(linesInOutput))
        
        for line in linesInOutput:
            if "function_call" in line:
                root = ET.fromstring(line)
                function = root.text
                
                nameOfFunctionToInvoke = function.split('(')[0]
                parameter = function.split('=')[1][1:-2]
        
        #print("tool executed")

        return AgentAction(
            #tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
            tool=nameOfFunctionToInvoke, tool_input=parameter, log=llm_output
        )
    
output_parser = CustomOutputParser()

### Content Handlers

Content Handlers are defined to convert the input to the input expected by the model and to extract the response from the JSON returned by the model. 

In [ ]:
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [ ]:
class LlamaContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generation"]



content_handler = ContentHandler()
llama_content_handler = LlamaContentHandler()


### Define the LLMs

We define the LLMs that we want to use for predictions. Ensure that you have access to Bedrock/Claude before you run this example

In [ ]:
modelArgs = {'max_tokens_to_sample': 250, 'temperature':0, "top_k":5,"top_p": 0.9,"stop_sequences":[]}
#modelArgs = {'max_tokens_to_sample': int(maxTokensToSample), 'temperature':float(temp), "top_k":int(topK),"top_p": float(topP),"stop_sequences":[]}
llm2 = Bedrock(model_id="anthropic.claude-v2",model_kwargs=modelArgs)



### Define the LLM Chain/Agent/AgentExecutor

The LLM chain is defined and we provide the LLM that we want to use along with the PromptTemplate that we want to use

In [ ]:
llm_chain = LLMChain(llm=llm2, prompt=claude_prompt)

tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    #llm_chain=llm_chain,
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\n\nHuman:"],
    allowed_tools=tool_names,
)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Inference

Now that we have defined the tools, the prompts and the agent, we can perform an inference. When you run the block below you should see an output like 

```
 <scratchpad>
I do not have enough information to generate a full TPS report for this question. However, I have been provided with the generateTpsReport function that can help me gather more details to answer this properly. I will call that function to get more information.
</scratchpad>

<function_call>generateTpsReport(topic="software engineer fixing 5 bugs")</function_call>
the tps report tool executed




Human:<function_result>here is a report about software engineer fixing 5 bugs. Write 5 sentences in a paragraph about software engineer fixing 5 bugs as a official status report </function_result>
 <answer>
Here is a TPS report about a software engineer fixing 5 bugs in a day:

The software engineer started the day by reviewing the bug tracking system and prioritizing the top 5 critical bugs reported by customers. The first bug was a crash that occurred when exporting data, which the engineer was able to quickly reproduce and fix in about an hour. The second and third bugs were related to formatting issues in the UI, which required some tweaks to the CSS styling to resolve. The fourth bug was a tricky logical error in the validation logic, needing careful debugging of the code flow. The fifth bug was a performance issue that required some optimization of the database queries. Through persistence and diligence, the software engineer was able to fix all 5 high priority bugs by the end of the day, improving the product and customer experience.
</answer>
```
A debug statement added to the tool which is printed to the console ```the tps report tool executed``` confirms that the tool was invoked


In [ ]:
langchain.debug = False
agent_executor.run("run TPS report for finishing all the timesheets")